# Vamos aprender a trabalhar com PDF usando o Python

- Regra geral: PDF foi feito justamente para bloquear muita coisa, então não é fácil "brincar" com um pdf
- Mesmo assim, Python tem várias bibliotecas que vão nos ajudar, vamos focar em 2:
    - PyPDF2
    - Tabula
- Ler e extrair informações de um PDF a gente consegue fazer.
- Escrever e Editar, aí já é outra história

### Para os nossos exemplos, vamos avaliar o Release de Resultados do 3º e 4º Trimestre de 2020 da Magazine Luiza

#### 1º Objetivo: Queremos conseguir separar apenas o DRE do Release de Resultados (Página 14) para enviar para a Diretoria, como fazemos?
    - Separar as páginas de um pdf

In [2]:
#Bibliotecas 
import PyPDF2 as pyf       #Mexer em pdfs
from pathlib import Path   #Mexer nos arquivos do pc

#Coisas para uzar
pyf.PdfFileReader   #Ler
pyf.PdfFileWriter   #criar novos e escrever neles
pyf.PdfFileMerger   #Mesclar varios

#Arquivo
nome_arquivo = 'MGLU_ER_3T20_POR.pdf'
arquivo_pdf = pyf.PdfFileReader(nome_arquivo)
                            
i = 1                    #vai por pagina
for pagina in arquivo_pdf.pages:
    novo_arquivo = pyf.PdfFileWriter()   #vai criar uma pagina vazia
    novo_arquivo.addPage(pagina)       #vai adicionar uma pagina criada
    #salvar o novo_pdf                      #modo de leitura e gravação
    with Path(f'Paginas/Pagina{i}.pdf').open(mode = 'wb') as arquivo_final:
        novo_arquivo.write(arquivo_final)
    
    i+=1

#### 2º Objetivo: Com o Release de Resultados já separado página por página, queremos incluir apenas as Páginas de Destaque (Página 1), DRE (Página 14) e Balanço (Página 16).
    - Juntar vários pdfs em 1

In [3]:
paginas = [1,14,16]

#criar um novo pdf
novo_pdf = pyf.PdfFileWriter()

for num_paginas in paginas:
    #pegar o arquivo num_pagina
    arquivo_pdf = f'Paginas/Pagina{num_paginas}.pdf'  #Vai pegar os arquivos em pdf
    ler_pdf = pyf.PdfFileReader(arquivo_pdf)  #Vai ler eles e criar um abjeto
    pagina = ler_pdf.pages[0]   #vai pegar tudo dentro do objeto 
    #adicionar ele no novo_pdf
    novo_pdf.addPage(pagina) 
    
    #salvar o novo_pdf
    with Path('Combinação.pdf').open(mode = 'wb') as arquivo_finall:
        novo_pdf.write(arquivo_finall)

### Extra: Para adicionar todas as páginas de 2 pdfs

In [4]:
juntar_pdfs = pyf.PdfFileMerger()

Arquivo1 = 'Pratica Simples/Comprovante de matricula.pdf'
Arquivo2 = 'Pratica Simples/Materias cursadas.pdf'

juntar_pdfs.append(Arquivo1, pages = (0,1))
juntar_pdfs.append(Arquivo2, pages = (0,1))

with Path('Pratica Simples/Juntados.pdf').open(mode = 'wb') as arquivo_final:
        juntar_pdfs.write(arquivo_final)

Superfluous whitespace found in object header b'1' b'0'
Superfluous whitespace found in object header b'2' b'0'
Superfluous whitespace found in object header b'3' b'0'
Superfluous whitespace found in object header b'19' b'0'
Superfluous whitespace found in object header b'1' b'0'
Superfluous whitespace found in object header b'2' b'0'
Superfluous whitespace found in object header b'3' b'0'
Superfluous whitespace found in object header b'17' b'0'
Superfluous whitespace found in object header b'16' b'0'
Superfluous whitespace found in object header b'14' b'0'
Superfluous whitespace found in object header b'4' b'0'
Superfluous whitespace found in object header b'15' b'0'
Superfluous whitespace found in object header b'13' b'0'
Superfluous whitespace found in object header b'12' b'0'
Superfluous whitespace found in object header b'11' b'0'
Superfluous whitespace found in object header b'10' b'0'
Superfluous whitespace found in object header b'9' b'0'
Superfluous whitespace found in object 

# Funcionalidades que podem ser úteis:

- Inserir arquivo no meio do outro
- Quero colocar dentro do Resultado do 4T20 os destaques do 3T20 para poder comparar os 2 dentro do mesmo relatório

In [8]:
mesclar_pdfs = pyf.PdfFileMerger()  #cria mas vazio

arquivo1 = 'MGLU_ER_3T20_POR.pdf'
arquivo2 = 'MGLU_ER_4T20_POR.pdf'

mesclar_pdfs.append(arquivo1)  #adiciona o pdf do 3 semestre
mesclar_pdfs.merge(1, arquivo2)   #adiciona no meio do anterior

with Path('PdfMerge.pdf').open(mode = 'wb') as arquivo_final:
        mesclar_pdfs.write(arquivo_final)

- Rodar Página

In [12]:
rodar_pdfs = pyf.PdfReader(arquivo1)  # leu o arquivos dos pdfs

pdf_final = pyf.PdfFileWriter()
for pagina in rodar_pdfs.pages:  #rodou dentro das paginas dele 
    pagina.rotateClockwise(90)   # girar 90
    pdf_final.addPage(pagina)
    
with Path('PdfRodado.pdf').open(mode = 'wb') as arquivo_final:
    pdf_final.write(arquivo_final)

# Trabalhando com Textos e Informações Dentro do PDF

#### 1º Objetivo: Quero identificar como foram as Despesas com Vendas da MGLU
    - Pegar texto da página e identificar onde está essa informação

#### 2º Objetivo: Quero analisar o DRE (sem ajuste - Página 5)
    - Para ler tabelas em pdf, use o tabula (é ninja)

#### 3º Objetivo: Quero analisar o Capital de Giro e os Investimentos (ambas as tabelas na página 12)
    - Páginas com mais de 1 tabela

#### O que fazer quando o tabula não consegue ler alguma linha da tabela? Como o cabeçalho, no nosso caso?

# Outro método que pode ser útil algum dia: Captar Imagem em um pdf
    - biblioteca pikepdf

In [ ]:
from pikepdf import Pdf, PdfImage

filename = "MGLU_ER_3T20_POR.pdf"
example = Pdf.open(filename)

n_arquivo = 0
for i, pagina in enumerate(example.pages):
    for nome, imagem in pagina.images.items():
        imagem_salvar = PdfImage(imagem)
        imagem_salvar.extract_to(fileprefix=f'imgs/Pagina_{i:02}_{n_arquivo}')
        n_arquivo += 1

# Substituir texto no pdf tipo contrato

- Não recomendo fazer diretamente pelo Python. Realmente do que vi a melhor opção me parece o Word fazer isso
- Caso precise automatizar, automatize o processo fazendo ele pelo Word
- Quem quiser MUITO fazer isso pelo Python, tem um link aqui que vai te ajudar de uma solução que achei que funciona. Tem seus bugs/cuidados especiais, mas funciona: https://pdf.co/samples/pdf-co-web-api-replace-text-from-pdf-python-replace-text-from-url